In [2]:
# Initial Imports 
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 

In [7]:
df_client_data = pd.read_csv('../HET/Data/client_data.csv')
df_client_membership = pd.read_csv('../HET/Data/client_membership.csv')
df_benchmark = pd.read_csv('../HET/Data/benchmark_data.csv')

/var/folders/h4/bmfyh9q95ws8cxksqm1z9m4c0000gn/T/ipykernel_74280/1195080869.py:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_client_data = pd.read_csv('../HET/Data/client_data.csv')
/var/folders/h4/bmfyh9q95ws8cxksqm1z9m4c0000gn/T/ipykernel_74280/1195080869.py:3: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_benchmark = pd.read_csv('../HET/Data/benchmark_data.csv')


In [40]:
df_benchmark.dtypes

gender                       object
age                           int64
uniq_mbr_cnt                 object
mbr_months_ccs               object
Primary_CCS_Dx_Level_1       object
Primary_CCS_Dx_Level_2       object
Total_Allowed                object
Total_Paid                   object
Allowed_Inpatient            object
Allowed_Outpatient           object
Allowed_Professional         object
Paid_Inpatient               object
Paid_Outpatient              object
Paid_Professional            object
Adm_cnt_Inpatient            object
Day_cnt_Inpatient            object
Visit_cnt_Outpatient         object
Visit_cnt_Professional       object
Proc_cnt_Professional        object
Svc_unit_cnt_Professional    object
non_utilizer                 object
dtype: object

In [41]:
#TODO It seems as though all data, besides Primary_CCS, should be in float format
# Might just drop Primary_CCS for some analysis here 